In [1]:
from __future__ import print_function
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..

fatal: destination path 'revcomp_experiments' already exists and is not an empty directory.
/users/hannahgz/revcomp_experiments/revcomp_experiments
running install
running bdist_egg
running egg_info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
reading manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.7/motif_density_and_position_sim.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scripts
changing mode of build/bd

In [2]:
!pip install concise
!pip install keras-genomics

In [3]:
!pip install git+https://github.com/kundajelab/simdna.git@v0.4.3.1#egg=simdna
!git clone https://github.com/kundajelab/revcomp_experiments.git
%cd revcomp_experiments/
!python setup.py install
%cd ..
import simdna

fatal: destination path 'revcomp_experiments' already exists and is not an empty directory.
/users/hannahgz/revcomp_experiments/revcomp_experiments
running install
running bdist_egg
running egg_info
writing revcompexp.egg-info/PKG-INFO
writing dependency_links to revcompexp.egg-info/dependency_links.txt
writing requirements to revcompexp.egg-info/requires.txt
writing top-level names to revcompexp.egg-info/top_level.txt
reading manifest file 'revcompexp.egg-info/SOURCES.txt'
writing manifest file 'revcompexp.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib

creating build/bdist.linux-x86_64/egg
creating build/bdist.linux-x86_64/egg/EGG-INFO
installing scripts to build/bdist.linux-x86_64/egg/EGG-INFO/scripts
running install_scripts
running build_scripts
creating build/bdist.linux-x86_64/egg/EGG-INFO/scripts
copying build/scripts-3.7/motif_density_and_position_sim.py -> build/bdist.linux-x86_64/egg/EGG-INFO/scripts
changing mode of build/bd

In [4]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [5]:
#@title
from __future__ import absolute_import, division, print_function
from collections import OrderedDict
import re
import gzip
import os
import json
from simdna import random


DEFAULT_LETTER_TO_INDEX = {'A': 0, 'C': 1, 'G': 2, 'T': 3}


DEFAULT_BACKGROUND_FREQ = OrderedDict(
    [('A', 0.3), ('C', 0.2), ('G', 0.2), ('T', 0.3)])


DEFAULT_DINUC_FREQ = OrderedDict([
 ('AA',0.095),
 ('AC',0.050),
 ('AG',0.071),
 ('AT',0.075),
 ('CA',0.073),
 ('CC',0.054),
 ('CG',0.010),
 ('CT',0.072),
 ('GA',0.060),
 ('GC',0.044),
 ('GG',0.054),
 ('GT',0.050),
 ('TA',0.064),
 ('TC',0.060),
 ('TG',0.073),
 ('TT',0.095),
])


def get_file_handle(filename,mode="r"):
    if (re.search('.gz$',filename) or re.search('.gzip',filename)):
        if (mode=="r"):
            mode="rb";
        elif (mode=="w"):
            #I think write will actually append if the file already
            #exists...so you want to remove it if it exists
            if os.path.isfile(filename):
                os.remove(filename);
        return gzip.open(filename,mode)
    else:
        return open(filename,mode) 


def default_tab_seppd(s):
    s = trim_newline(s)
    s = split_by_delimiter(s, "\t")
    return s


def trim_newline(s):
    return s.rstrip('\r\n')


def split_by_delimiter(s, delimiter):
    return s.split(delimiter)


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
        line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();


def process_line(line, i, ignore_input_title,
                 transformation, action, progress_update=None):
    if (i > 1 or (ignore_input_title==False)):
        action(transformation(line),i)


def print_progress(progress_update, i, file_name=None):
    if progress_update is not None:
        if (i%progress_update == 0):
            print ("Processed "+str(i)+" lines"
                   +str("" if file_name is None else " of "+file_name))


class VariableWrapper():
    """ For when I want reference-type access to an immutable"""
    def __init__(self, var):
        self.var = var   


def enum(**enums):
    class Enum(object):
        pass
    to_return = Enum
    for key,val in enums.items():
        if hasattr(val, '__call__'): 
            setattr(to_return, key, staticmethod(val))
        else:
            setattr(to_return, key, val)
    to_return.vals = [x for x in enums.values()]
    to_return.the_dict = enums
    return to_return


def combine_enums(*enums):
    new_enum_dict = OrderedDict()
    for an_enum in enums:
        new_enum_dict.update(an_enum.the_dict)
    return enum(**new_enum_dict)


def sampleFromProbsArr(arrWithProbs):
    """Samples from a discrete distribution.
    Arguments:
        arrWithProbs: array of probabilities
    Returns:
        an index, sampled with the probability of that index in
    array of probabilities.
    """
    randNum = random.random()
    cdfSoFar = 0
    for (idx, prob) in enumerate(arrWithProbs):
        cdfSoFar += prob
        if (cdfSoFar >= randNum or idx == (len(arrWithProbs) - 1)):  # need the
            # letterIdx==(len(row)-1) clause because of potential floating point errors
            # that mean arrWithProbs doesn't sum to 1
            return idx


reverseComplementLookup = {'A': 'T', 'T': 'A', 'G': 'C', 'C': 'G',
                           'a': 't', 't': 'a', 'g': 'c', 'c': 'g', 'N': 'N', 'n': 'n'}


def reverseComplement(sequence):
    reversedSequence = sequence[::-1]
    reverseComplemented = "".join(
        [reverseComplementLookup[x] for x in reversedSequence])
    return reverseComplemented


def sampleWithoutReplacement(arr, numToSample):
    arrayCopy = [x for x in arr]
    for i in range(numToSample):
        randomIndex = int(random.random() * (len(arrayCopy) - i)) + i
        swapIndices(arrayCopy, i, randomIndex)
    return arrayCopy[0:numToSample]


def swapIndices(arr, idx1, idx2):
    temp = arr[idx1]
    arr[idx1] = arr[idx2]
    arr[idx2] = temp


def get_file_name_parts(file_name):
    p = re.compile(r"^(.*/)?([^\./]+)(\.[^/]*)?$")
    m = p.search(file_name)
    return FileNameParts(m.group(1), m.group(2), m.group(3))


class FileNameParts(object):

    def __init__(self, directory, core_file_name, extension):
        self.directory = directory if (directory is not None) else os.getcwd()
        self.core_file_name = core_file_name
        self.extension = extension

    def get_full_path(self):
        return self.directory+"/"+self.file_name

    def get_core_file_name_and_extension(self):
        return self.core_file_name+self.extension

    def get_transformed_core_file_name(self, transformation, extension=None):
        to_return = transformation(self.core_file_name)
        if (extension is not None):
            to_return = to_return + extension
        else:
            if (self.extension is not None):
                to_return = to_return + self.extension
        return to_return

    def get_transformed_file_path(self, transformation, extension=None):
        return (self.directory+"/"+
                self.get_transformed_core_file_name(transformation,
                                                    extension=extension))


def format_as_json(jsonable_data):
    return json.dumps(jsonable_data, indent=4, separators=(',', ': '))


class ArgumentToAdd(object):
    """
        Class to append runtime arguments to a string
        to facilitate auto-generation of output file names.
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = (','.join([str(el) for el in self.val])\
                   if (isinstance(self.val, str)==False and
                       hasattr(self.val,"__len__")) else str(self.val))
        return self.argNamePrefix()+string;
        # return self.argNamePrefix()+str(self.val).replace(".","p");


class FloatArgument(ArgumentToAdd):
    """
       Replace the period with a p 
    """
    def __init__(self, val, argumentName=None, argNameAndValSep="-"):
        self.val = val;
        self.argumentName = argumentName;
        self.argNameAndValSep = argNameAndValSep;
    def argNamePrefix(self):
        return ("" if self.argumentName is None else self.argumentName+str(self.argNameAndValSep))
    def transform(self):
        string = str(self.val)
        string = string.replace(".","p")
        return self.argNamePrefix()+string


class BooleanArgument(ArgumentToAdd):

    def transform(self):
        assert self.val  # should be True if you're calling transformation
        return self.argumentName


class CoreFileNameArgument(ArgumentToAdd):

    def transform(self):
        import fileProcessing as fp
        return self.argNamePrefix() + fp.getCoreFileName(self.val)


class ArrArgument(ArgumentToAdd):

    def __init__(self, val, argumentName, sep="+", toStringFunc=str):
        super(ArrArgument, self).__init__(val, argumentName)
        self.sep = sep
        self.toStringFunc = toStringFunc

    def transform(self):
        return self.argNamePrefix() + self.sep.join([self.toStringFunc(x) for x in self.val])


class ArrOfFileNamesArgument(ArrArgument):

    def __init__(self, val, argumentName, sep="+"):
        import fileProcessing as fp
        super(ArrOfFileNamesArgument, self).__init__(val, argumentName,
                                                     sep, toStringFunc=lambda x: fp.getCoreFileName(x))


def addArguments(string, args, joiner="_"):
    """
        args is an array of ArgumentToAdd.
    """
    for arg in args:
        string = string + ("" if arg.val is None or arg.val is False or (hasattr(
            arg.val, "__len__") and len(arg.val) == 0) else joiner + arg.transform())
    return string
class Embedding(object):
    """Represents something that has been embedded in a sequence.
    Think of this as a combination of an embeddable + a start position.
    Arguments:
        what: object representing the thing that has been embedded.\
            Should have`` __str__`` and ``__len__`` defined.\
            Often is an instance of :class:`.AbstractEmbeddable`
        startPos: int, the position relative to the start of the parent\
            sequence at which seq has been embedded
    """

    def __init__(self, what, startPos):
        self.what = what
        self.startPos = startPos

    def __str__(self):
        return "pos-" + str(self.startPos) + "_" + str(self.what)

    @classmethod
    def fromString(cls, string, whatClass=None):
        """Recreate an :class:`.Embedding` object from a string.
        Arguments:
            string: assumed to have format:\
                ``description[-|_]startPos[-|_]whatString``, where
                ``whatString`` will be provided to ``whatClass``
            whatClass: the class (usually a :class:`.AbstractEmbeddable`) that\
                will be used to instantiate the what from the whatString
        Returns:
            The Embedding class called with
            ``what=whatClass.fromString(whatString)`` and
            ``startPos=int(startPos)``
        """
        if (whatClass is None):
            from simdna.synthetic.embeddables import StringEmbeddable
            whatClass = StringEmbeddable
        # was printed out as pos-[startPos]_[what], but the
        #[what] may contain underscores, hence the maxsplit
        # to avoid splitting on them.
        p = re.compile(r"pos\-(\d+)_(.*)$")
        m = p.search(string)
        startPos = m.group(1)
        whatString = m.group(2) 
        return cls(what=whatClass.fromString(whatString),
                   startPos=int(startPos))
      
def getEmbeddingsFromString(string):
    """Get a series of :class:`.Embedding` objects from a string.
    
    Splits the string on commas, and then passes the comma-separated vals
        to :func:`.Embedding.fromString`
    Arguments:
        string: The string to turn into an array of Embedding objects
    Returns:
        an array of :class:`.Embedding` objects
    """
    if len(string) == 0:
        return []
    else:
        embeddingStrings = string.split(",")
        return [Embedding.fromString(x) for x in embeddingStrings]
      
def read_simdata_file(simdata_file, one_hot_encode=False, ids_to_load=None):
    ids = []
    sequences = []
    embeddings = []
    labels = []
    if (ids_to_load is not None):
        ids_to_load = set(ids_to_load)
    def action(inp, line_number):
        if (line_number > 1):
            if (ids_to_load is None or (inp[0] in ids_to_load)):
                ids.append(inp[0]) 
                sequences.append(inp[1])
                embeddings.append(getEmbeddingsFromString(inp[2]))
                labels.append([int(x) for x in inp[3:]])
    perform_action_on_each_line_of_file(
        file_handle=get_file_handle(simdata_file),
        action=action,
        transformation=default_tab_seppd)
    return enum(
            ids=ids,
            sequences=sequences,
            embeddings=embeddings,
            labels=np.array(labels))
  


def perform_action_on_each_line_of_file(
    file_handle
    #should be a function that accepts the
    #preprocessed/filtered line and the line number
    , action
    , transformation=default_tab_seppd
    , ignore_input_title=False
    , progress_update=None
    , progress_update_file_name=None):

    i = 0;
    for line in file_handle:
        i += 1;
#         line = line.decode("utf-8")
        process_line(line, i, ignore_input_title,
                     transformation, action, progress_update)
        print_progress(progress_update, i, progress_update_file_name)

    file_handle.close();

In [6]:
import keras 
import keras_genomics
import numpy as np

from keras import backend as K 
from keras.layers.core import Dropout 
from keras.layers.core import Flatten
from keras.engine import Layer
from keras.models import Sequential 
import keras.layers as k1
from keras.engine.base_layer import InputSpec

from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [7]:
# !motif_density_and_position_sim.py --seed 1234 --motifNames GATA_disc1 TAL1_known1 --max-motifs 4 --min-motifs 1 --mean-motifs 2 --zero-prob 0.5 --seqLength 1000 --posSdevInBp 100 --numSeqs 20000

In [8]:
# !head DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-20000.simdata

In [44]:
sequences = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").sequences
embeddings = read_simdata_file("DensityEmbedding_motifs-GATA_disc1+TAL1_known1_min-1_max-4_mean-2_zeroProb-0p5_seqLength-1000_posSdevInBp-100_numSeqs-10000.simdata").embeddings

In [45]:
def check(embeddings): 
    if len(embeddings) > 0:
        all_chars = ""
        for x in embeddings: 
            all_chars += str(x.what)
        if 'TAL' in all_chars and 'GATA' in all_chars: 
            return [1, 1]
        if 'TAL' in all_chars and 'GATA' not in all_chars:
            return [1, 0]
        if 'TAL' not in all_chars and 'GATA' in all_chars:
            return [0, 1]
    return [0,0]

In [46]:
y= np.zeros((0,2))
for x in embeddings: 
    thing = check(x)
    y = np.append(y, [thing], axis=0)

In [47]:
import numpy as np

# def mutate(s, num, target):
#     change_locs = set(sample(range(len(s)), num))
#     changed = (target if i in change_locs else c for i,c in enumerate(s))
#     return ''.join(changed)

def mutate(y, mutation_prob):
    np.random.seed(1234)
    mutated_y = []
    for row in y:
        new_labels_for_row = []
        for label in row:
            if np.random.uniform() < mutation_prob:
                new_labels_for_row.append(1-label)
            else:
                new_labels_for_row.append(label)
        mutated_y.append(new_labels_for_row)
    return np.array(mutated_y)

In [48]:
ltrdict = {'a':[1,0,0,0],
           'c':[0,1,0,0],
           'g':[0,0,1,0],
           't':[0,0,0,1],
           'n':[0,0,0,0],
           'A':[1,0,0,0],
           'C':[0,1,0,0],
           'G':[0,0,1,0],
           'T':[0,0,0,1],
           'N':[0,0,0,0]}

def onehot_encode(sequence):
    return np.array([ltrdict[x] for x in sequence])

x = np.array([onehot_encode(x) for x in sequences]) 

In [49]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.25, random_state = 42)

In [50]:
x_test[0][1]

array([0, 0, 0, 1])

In [51]:
noisy_y_train = mutate(y_train, mutation_prob=0.2)

In [52]:
x_train_augment = np.asarray([val for val in x_train for __ in (0,1)])
for i in range(len(x_train_augment)):
    if i % 2 == 1:
        x_train_augment[i] = np.flip(x_train_augment[i])

x_test_augment = np.asarray([val for val in x_test for __ in (0,1)])
for i in range(len(x_test_augment)):
    if i % 2 == 1:
        x_test_augment[i] = np.flip(x_test_augment[i])

In [53]:
noisy_y_train_augment = np.asarray([val for val in noisy_y_train for __ in (0,1)])
y_test_augment = np.asarray([val for val in y_test for __ in (0,1)])

In [54]:
class RevCompSumPool(Layer): 
    def __init__(self, **kwargs): 
        super(RevCompSumPool, self).__init__(**kwargs)

    def build(self, input_shape):
        self.num_input_chan = input_shape[2]
        super(RevCompSumPool, self).build(input_shape)

    def call(self, inputs): 
        #divide by sqrt 2 for variance preservation
        inputs = (inputs[:,:,:int(self.num_input_chan/2)] + inputs[:,:,int(self.num_input_chan/2):][:,::-1,::-1])/(1.41421356237)
        return inputs
      
    def compute_output_shape(self, input_shape):
        return (input_shape[0], input_shape[1], int(input_shape[2]/2))


In [106]:
kernel_size = 15
filters= 15
input_length = 1000

In [144]:
seed_num = 6000

from numpy.random import seed
from tensorflow import set_random_seed
from keras.callbacks import EarlyStopping, History, ModelCheckpoint

seed(seed_num)
set_random_seed(seed_num)

In [145]:
from keras.layers import Input
from keras.models import Model
from keras.models import load_model

In [146]:
filename = ('Seed %s.txt' % seed_num, str(seed_num))[0]
f = open(filename, 'w')

In [147]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score

def evaluate(model, x, y): 
    y_pred = model.predict(x)
    auroc = roc_auc_score(y, model.predict(x)) 
    auprc = average_precision_score(y, model.predict(x))
    print("auroc: " + str(auroc))
    print("auprc: "+ str(auprc))
    f.write("auroc: " + str(auroc) + "\n")
    f.write("auprc: " + str(auprc) + "\n")

In [148]:
from keras.initializers import Initializer
def _compute_fans(shape, data_format='channels_last'):
    """Computes the number of input and output units for a weight shape.
    # Arguments
        shape: Integer shape tuple.
        data_format: Image data format to use for convolution kernels.
            Note that all kernels in Keras are standardized on the
            `channels_last` ordering (even when inputs are set
            to `channels_first`).
    # Returns
        A tuple of scalars, `(fan_in, fan_out)`.
    # Raises
        ValueError: in case of invalid `data_format` argument.
    """
    if len(shape) == 2:
        fan_in = shape[0]
        fan_out = shape[1]
    elif len(shape) in {3, 4, 5}:
        # Assuming convolution kernels (1D, 2D or 3D).
        # TH kernel shape: (depth, input_depth, ...)
        # TF kernel shape: (..., input_depth, depth)
        if data_format == 'channels_first':
            receptive_field_size = np.prod(shape[2:])
            fan_in = shape[1] * receptive_field_size
            fan_out = shape[0] * receptive_field_size
        elif data_format == 'channels_last':
            receptive_field_size = np.prod(shape[:-2])
            fan_in = shape[-2] * receptive_field_size
            fan_out = shape[-1] * receptive_field_size
        else:
            raise ValueError('Invalid data_format: ' + data_format)
    else:
        # No specific assumptions.
        fan_in = np.sqrt(np.prod(shape))
        fan_out = np.sqrt(np.prod(shape))
    return fan_in, fan_out

class RevcompVarianceScaling(Initializer):
    def __init__(self, scale=1.0,
                 mode='fan_in',
                 distribution='normal',
                 seed=None):
        if scale <= 0.:
            raise ValueError('`scale` must be a positive float. Got:', scale)
        mode = mode.lower()
        if mode not in {'fan_in', 'fan_out', 'fan_avg'}:
            raise ValueError('Invalid `mode` argument: '
                             'expected on of {"fan_in", "fan_out", "fan_avg"} '
                             'but got', mode)
        distribution = distribution.lower()
        if distribution not in {'normal', 'uniform'}:
            raise ValueError('Invalid `distribution` argument: '
                             'expected one of {"normal", "uniform"} '
                             'but got', distribution)
        self.scale = scale
        self.mode = mode
        self.distribution = distribution
        self.seed = seed

    def __call__(self, shape, dtype=None):
        fan_in, fan_out = _compute_fans(shape)
        fan_out = fan_out*2 #revcomp kernel underestimates fan_out
        print("fanin:",fan_in, "fanout:",fan_out, self.scale, self.mode)
        scale = self.scale
        if self.mode == 'fan_in':
            scale /= max(1., fan_in)
        elif self.mode == 'fan_out':
            scale /= max(1., fan_out)
        else:
            scale /= max(1., float(fan_in + fan_out) / 2)
        if self.distribution == 'normal':
            # 0.879... = scipy.stats.truncnorm.std(a=-2, b=2, loc=0., scale=1.)
            stddev = np.sqrt(scale) / .87962566103423978
            return K.truncated_normal(shape, 0., stddev,
                                      dtype=dtype, seed=self.seed)
        else:
            limit = np.sqrt(3. * scale)
            return K.random_uniform(shape, -limit, limit,
                                    dtype=dtype, seed=self.seed)

    def get_config(self):
        return {
            'scale': self.scale,
            'mode': self.mode,
            'distribution': self.distribution,
            'seed': self.seed
        }

# Reverse Complement

## Augmented

In [149]:
augment_model = keras.models.Sequential()
augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
                        input_shape=(input_length,4), padding="same"))
# reg_model.add(k1.BatchNormalization())
augment_model.add(k1.core.Activation("relu"))
# augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
#                         padding="same"))
# reg_model.add(k1.BatchNormalization())
# augment_model.add(k1.core.Activation("relu"))
# augment_model.add(k1.Conv1D(filters=filters, kernel_size=kernel_size,
#                         padding="same"))
# # reg_model.add(k1.BatchNormalization())
# augment_model.add(k1.core.Activation("relu"))
augment_model.add(k1.pooling.MaxPooling1D(pool_size=40,padding="same",
                                               strides=40))
augment_model.add(Flatten())
# augment_model.add(k1.Dense(units = 100, activation = "relu"))
augment_model.add(k1.Dense(units = 2))
augment_model.add(k1.core.Activation("sigmoid"))

In [150]:
augment_model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
early_stopping_callback = keras.callbacks.EarlyStopping(
                              monitor='val_loss',
                              patience= 40,
                              restore_best_weights=True)
history_augment = augment_model.fit(x_train_augment, noisy_y_train_augment, validation_split = 0.2, 
                           callbacks=[early_stopping_callback], batch_size = 200, epochs = 100)

Train on 12000 samples, validate on 3000 samples
Epoch 1/100
12000/12000 [==============================] - 2s 193us/step - loss: 0.7035 - acc: 0.4946 - val_loss: 0.6957 - val_acc: 0.5003
Epoch 2/100
12000/12000 [==============================] - 1s 55us/step - loss: 0.6964 - acc: 0.5020 - val_loss: 0.6952 - val_acc: 0.5065
Epoch 3/100
12000/12000 [==============================] - 1s 50us/step - loss: 0.6945 - acc: 0.5151 - val_loss: 0.6957 - val_acc: 0.5142
Epoch 4/100
12000/12000 [==============================] - 1s 53us/step - loss: 0.6914 - acc: 0.5245 - val_loss: 0.6943 - val_acc: 0.5115
Epoch 5/100
12000/12000 [==============================] - 1s 54us/step - loss: 0.6906 - acc: 0.5299 - val_loss: 0.6951 - val_acc: 0.5107
Epoch 6/100
12000/12000 [==============================] - 1s 53us/step - loss: 0.6873 - acc: 0.5433 - val_loss: 0.6924 - val_acc: 0.5202
Epoch 7/100
12000/12000 [==============================] - 1s 56us/step - loss: 0.6857 - acc: 0.5531 - val_loss: 0.6954 - 

In [151]:
augment_model.set_weights(early_stopping_callback.best_weights)
augment_filename = ('augment_%s.h5' % seed_num, str(seed_num))[0]
augment_model.save(augment_filename)
augment_final = load_model(augment_filename)

# Evaluating Models

In [152]:
print("Augmented Model")
f.write("Augment Model\n")
evaluate(augment_model, x_test_augment, y_test_augment)

Augmented Model
auroc: 0.8929991031253879
auprc: 0.9063214648145622


In [153]:
f.close()